In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

# Get the current research's directory
research_dir = os.path.dirname(os.path.abspath('__file__'))

# Move one directory back
parent_dir = os.path.dirname(research_dir)

# Change the current working directory to the parent directory
os.chdir(parent_dir)

# Print the current working directory to confirm
print(f"Current working directory: {os.getcwd()}")

# OakLang Agent HITL + Adaptative/Agentic RAG

## Agents/Graphs as simple tools

In [ ]:
## COMPILE THE GRAPHS
from frank.workflow_builder import WorkflowBuilder
from frank.config.layouts.adaptative_rag_config_graph import AdaptativeRAGConfigGraph
from frank.config.layouts.oak_human_loop_config_graph import OakHumanLoopConfigGraph
from frank.models.stategraph.ragstategraph import RAGState
from frank.models.stategraph.stategraph import SharedState
from frank.utils.common import read_yaml
from frank.utils.logger import setup_logging
from frank.constants import *

## Read the config.yaml
config = read_yaml(CONFIG_FILE_PATH)

## Setup logging Configuration
setup_logging(config)

## Workflow Configuration for the main graph
workflow_builder = WorkflowBuilder(
    config=AdaptativeRAGConfigGraph, 
    state_schema=RAGState,
)
ADAPTATIVE_RAG_GRAPH = workflow_builder.compile() # compile the graph

## Workflow Configuration for the main graph
workflow_builder = WorkflowBuilder(
    config=OakHumanLoopConfigGraph, 
    state_schema=SharedState,
)
OAKLANG_AGENT_GRAPH = workflow_builder.compile() # compile the graph

In [ ]:
## BUILD THE SIMPLE TOOLS
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

from services.ai_foundry.llm import LLMServices

LLMServices.launch()

@tool
async def oaklang_agent_tool(input: str) -> str:
    """Tool to ask the evolution or random movements of certain Pokémon. The input is a question."""
    message_input = {"messages": [{"role": "human", "content": input}]}
    response = await OAKLANG_AGENT_GRAPH.ainvoke(message_input)
    return response['messages'][-1].content

@tool
async def adaptive_rag_tool(input: str) -> str:
    """Tool to use RAG about Pokémon series questions. The input is a question."""
    message_input = {"messages": [{"role": "human", "content": input}]}
    response = await ADAPTATIVE_RAG_GRAPH.ainvoke(message_input)
    return response['messages'][-1].content

tools = [oaklang_agent_tool, adaptive_rag_tool]

agent_executor = create_react_agent(model = LLMServices.model, tools=tools, prompt="You are an agent that can route into tools")

async def main():
    return await agent_executor.ainvoke({
        "messages": [{
            "role": "human",
            "content": "In the third chapter of the Pokémon series, what is the name of the Pokémon that was flying in the daytime sky and subsequently caught? and what is the evolution of that pokemon?"
        }]
    })

response = await main()


In [ ]:
response

In [ ]:
response["messages"][-1].content

## Agents/Graphs as MCP tools

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

from services.ai_foundry.llm import LLMServices

LLMServices.launch()

client = MultiServerMCPClient(
    {
        "OakLangAgent": {
            "command": "python",
            "args": ["src/services/mcp/server_oaklang_agent.py"],
            "transport": "stdio",
            },
        "AdaptativeRAG": { # NOTE: python3 src/services/mcp/server_adaptative_rag.py
            "url": "http://localhost:8000/mcp",
            "transport": "streamable_http",
        },
        }
)
tools = await client.get_tools()
agent = create_react_agent(
    LLMServices.model,
    tools
)


response = await agent.ainvoke(
    {"messages": [{
        "role": "human",
        "content": "In the third chapter of the Pokémon series, what is the name of the Pokémon that was flying in the daytime sky and subsequently caught? and what is the evolution of that pokemon?"
        }]}
)


In [ ]:
response

In [ ]:
response["messages"][-1].content